In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [3]:
stock = "GOOGL"
df = pd.DataFrame(yf.download(stock, '2010-01-01', '2022-12-31')["Close"])

[*********************100%***********************]  1 of 1 completed


In [4]:
def rolling(lag, data):
    A = []
    df = data.tolist()
    for i in range(lag, len(df)):
        A.append(df[i-lag:i]) 
    return pd.DataFrame(A)

def UoD(x):
    if x<0:
        return -1
    if x>0:
        return 1
    else: 
        return 0

def NN(x,y, input_dim, out_dim):
		model = Sequential()
		model.add(Dense(100, input_dim = input_dim, activation="sigmoid"))
		model.add(Dense(100, input_dim = 100, activation="sigmoid"))
		model.add(Dense(100, input_dim = 100, activation="sigmoid"))
		model.add(Dense(out_dim, input_dim = 5, activation="softmax"))

		optimizer=Adam()
		model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

		model.fit(x, y, epochs=200, verbose=2)
		return model

In [5]:
df["returns"] =df["Close"].pct_change()
df["UoD"]= df["returns"].map(UoD)
df.dropna(inplace=True)

In [6]:
lag = 3
A = rolling(lag, df["returns"])
b = df["UoD"][lag::]

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(b).reshape(-1, 1)).toarray()
deon, dete, keyon, keyte = train_test_split(A.values,y, test_size=0.5)

In [7]:
a = NN(deon,keyon,lag,3)

Epoch 1/200
52/52 - 1s - loss: 0.7254 - accuracy: 0.4884 - 688ms/epoch - 13ms/step
Epoch 2/200
52/52 - 0s - loss: 0.6982 - accuracy: 0.5018 - 99ms/epoch - 2ms/step
Epoch 3/200
52/52 - 0s - loss: 0.6977 - accuracy: 0.5190 - 97ms/epoch - 2ms/step
Epoch 4/200
52/52 - 0s - loss: 0.6975 - accuracy: 0.5067 - 94ms/epoch - 2ms/step
Epoch 5/200
52/52 - 0s - loss: 0.6973 - accuracy: 0.5165 - 106ms/epoch - 2ms/step
Epoch 6/200
52/52 - 0s - loss: 0.6975 - accuracy: 0.5177 - 125ms/epoch - 2ms/step
Epoch 7/200
52/52 - 0s - loss: 0.6951 - accuracy: 0.5239 - 100ms/epoch - 2ms/step
Epoch 8/200
52/52 - 0s - loss: 0.6994 - accuracy: 0.5104 - 161ms/epoch - 3ms/step
Epoch 9/200
52/52 - 0s - loss: 0.7002 - accuracy: 0.5141 - 154ms/epoch - 3ms/step
Epoch 10/200
52/52 - 0s - loss: 0.6999 - accuracy: 0.5153 - 114ms/epoch - 2ms/step
Epoch 11/200
52/52 - 0s - loss: 0.7067 - accuracy: 0.5006 - 119ms/epoch - 2ms/step
Epoch 12/200
52/52 - 0s - loss: 0.7045 - accuracy: 0.4994 - 102ms/epoch - 2ms/step
Epoch 13/200
52

In [8]:
a.evaluate(a.predict(dete),keyte)

52/52 [==============================] - 1s 3ms/step - loss: 0.7103 - accuracy: 0.5269


[0.7102971076965332, 0.5269277691841125]